### Define jobname

In [1]:
pdbid = '2jpz'
drug  = None
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid
indexfile=f'{jobname}.simulate.ndx'

### Center protein for a solvated trajectory and put residues in the unit cell 
- Input and output are both solvated trajectories
- 1. make residues in the unit cell
- The sequence of commands here are used to generate the "process.solvated.traj" shell scripts for production runs

In [2]:
def rmpbc(filename,indexfile='index.ndx',center=0,output=0, postfix='c'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s {file}.tpr -f {filename} -o {file}_{postfix}.{extend} -pbc res -center -boxcenter zero -n {indexfile} -ndec 5 -ur compact'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##")
    !{cmd1}

### Output non-water parts with best fit 
- The index file needs to be setup first
- The sequence of commands here are used to generate the "rms.traj" shell scripts for production runs

In [3]:
def grep_and_fit(filename,indexfile='index.ndx',center=0,output=0, postfix='rms'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s rms.tpr -f {filename} -o {file}_{postfix}.{extend} -n {indexfile} -fit rot+trans -ndec 5'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##") 
    !{cmd1}

In [4]:
def rmpdbc_and_grep_and_fit(i):
    rmpbc(f"md{i}.xtc", indexfile, center='DNA')
    grep_and_fit(f"md{i}_c.xtc", indexfile, center='DNA', output='Model')
    grep_and_fit(f"md{i}_c.xtc", indexfile, center='DNA', output='non-Water', postfix='noh')
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(10):
        futures.append(executor.submit(rmpdbc_and_grep_and_fit, i))
    for future in concurrent.futures.as_completed(futures):
        pass   

##========================command=line========================##
echo "DNA 0" | gmx trjconv -s md0.tpr -f md0.xtc -o md0_c.xtc -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact
##======================end=command=line======================##
##========================command=line========================##
echo "DNA 0" | gmx trjconv -s md1.tpr -f md1.xtc -o md1_c.xtc -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact
##======================end=command=line======================##
##========================command=line========================##
echo "DNA 0" | gmx trjconv -s md2.tpr -f md2.xtc -o md2_c.xtc -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact
##======================end=command=line======================##
##========================command=line========================##
echo "DNA 0" | gmx trjconv -s md3.tpr -f md3.xtc -o md3_c.xtc -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact
#

### Cat rms trajectories
- The following is for heating and equilibration parts
- Use cat.traj to combine production run trajectories

In [5]:
!gmx trjcat -f md*_rms.xtc -o {jobname}_5us500kf.xtc -settime 0
!gmx trjconv -f {jobname}_5us500kf.xtc -o {jobname}_5us5kf.xtc -skip 100 -ndec 5

                      :-) GROMACS - gmx trjcat, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/2.simulation
Command line:
  gmx trjcat -f md0_rms.xtc md1_rms.xtc md2_rms.xtc md3_rms.xtc md4_rms.xtc md5_rms.xtc md6_rms.xtc md7_rms.xtc md8_rms.xtc md9_rms.xtc -o 2jpz_5us500kf.xtc -settime 0

Reading frame       1 time 4500010.000   

Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
              md0_rms.xtc        0.000 ps       10.000 ps
              md1_rms.xtc   500000.000 ps       10.000 ps
              md2_rms.xtc   1000000.000 ps       10.000 ps
              md3_rms.xtc   1500000.000 ps       10.000 ps
              md4_rms.xtc   2000000.000 ps       10.000 ps
              md5_rms.xtc   2500000.000 ps       10.000 ps
              md6_rms.xt

In [6]:
!gmx trjcat -f md*_noh.xtc -o {jobname}_5us500kf_noh.xtc -settime 0
!gmx trjconv -f {jobname}_5us500kf_noh.xtc -o {jobname}_5us5kf_noh.xtc -skip 100 -ndec 5

                      :-) GROMACS - gmx trjcat, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/2.simulation
Command line:
  gmx trjcat -f md0_noh.xtc md1_noh.xtc md2_noh.xtc md3_noh.xtc md4_noh.xtc md5_noh.xtc md6_noh.xtc md7_noh.xtc md8_noh.xtc md9_noh.xtc -o 2jpz_5us500kf_noh.xtc -settime 0

Reading frame       1 time 4500010.000   

Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
              md0_noh.xtc        0.000 ps       10.000 ps
              md1_noh.xtc   500000.000 ps       10.000 ps
              md2_noh.xtc   1000000.000 ps       10.000 ps
              md3_noh.xtc   1500000.000 ps       10.000 ps
              md4_noh.xtc   2000000.000 ps       10.000 ps
              md5_noh.xtc   2500000.000 ps       10.000 ps
              md6_no

### Move data to next folder

In [7]:
!cp {jobname}_5us500kf.xtc ../3.trajectory_data/
!cp {jobname}_5us5kf.xtc   ../3.trajectory_data/

## Visualize
```bash
pymol md0_c.pdb   md0_c.xtc
pymol md0_rms.pdb md0_rms.xtc
```